# Create dataset for BART

In [1]:
import numpy as np
import math
from matchms.importing import load_from_msp
from matchms.filtering import normalize_intensities

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from rdkit.Chem import PandasTools
from pandas import DataFrame as df
from rdkit import Chem
from rdkit.Chem.inchi import MolToInchiKey

import os
import spectrum_utils.plot as sup
import spectrum_utils.spectrum as sus
from collections import Counter
import random

#####

from tokenizers import Tokenizer
from tokenizers.models import Model
import torch
from collections import defaultdict
from typing import Union, Dict, List

In [2]:
# !pip install pandas==1.3.0

In [9]:
data_size = "1K"
prepared_data_path = f"/storage/brno2/home/ahajek/Spektro/MassGenie/data/trial_set/{data_size}_bart_prepared_data_test.pkl"


In [10]:
df = pd.read_pickle(prepared_data_path)

In [11]:
df

,destereo_smiles,input_ids,decoder_input_ids,encoder_attention_mask,decoder_attention_mask,labels,position_ids
258,CCOC(=O)Cn1c(CN2CCOCC2)nc2c1c(=O)n(c(=O)n2C)C,"[18, 27, 29, 30, 33, 34, 38, 39, 40, 41, 42, 4...","[503, 531, 531, 542, 531, 506, 510, 542, 507, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 531, 542, 531, 506, 510, 542, 507, ...","[0, 0, 5, 4, 4, 3, 4, 6, 6, 8, 9, 7, 7, 6, 3, ..."
23,OC(Cn1cnc(c2c1nc(n2)N1CCOCC1)N)CN1CCOCC1,"[40, 41, 42, 43, 44, 45, 50, 51, 52, 53, 54, 5...","[503, 542, 531, 506, 531, 565, 520, 555, 565, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 531, 506, 531, 565, 520, 555, 565, ...","[3, 7, 7, 6, 6, 5, 1, 3, 4, 5, 6, 6, 8, 5, 3, ..."
312,CCS(=O)(=O)N1CCN(C2C1CS(=O)(=O)C2)C(=O)CN(C)C,"[27, 36, 39, 40, 41, 42, 43, 44, 45, 46, 48, 4...","[503, 531, 531, 545, 506, 510, 542, 507, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 531, 545, 506, 510, 542, 507, 506, ...","[1, 1, 3, 3, 7, 8, 6, 6, 6, 2, 5, 1, 2, 4, 6, ..."
246,CN(S(=O)(=O)CC1COCC1NC(=O)Cn1cc(C)c(nc1=O)O)C,"[15, 26, 27, 28, 29, 30, 31, 33, 34, 36, 37, 3...","[503, 531, 541, 506, 545, 506, 510, 542, 507, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 541, 506, 545, 506, 510, 542, 507, ...","[4, 3, 6, 7, 6, 6, 6, 3, 1, 4, 4, 6, 8, 7, 8, ..."
56,COCCN1CCC(C1)CNS(=O)(=O)c1cn(C)c(=O)n(c1=O)C,"[33, 34, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4...","[503, 531, 542, 531, 531, 541, 520, 531, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 542, 531, 531, 541, 520, 531, 531, ...","[5, 1, 6, 6, 8, 9, 8, 8, 9, 5, 1, 5, 0, 4, 4, ..."
498,CCn1ncc(c1)CC(=O)N1CC(C(C1)NC(=O)CCc1n[nH]cn1)O,"[33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 4...","[503, 531, 531, 565, 520, 565, 555, 555, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 531, 565, 520, 565, 555, 555, 506, ...","[3, 0, 1, 4, 7, 6, 9, 9, 8, 9, 6, 4, 4, 5, 7, ..."
414,Cc1cnc(cn1)C(=O)N1CCN(C2C1CS(=O)(=O)C2)S(=O)(=O)C,"[27, 28, 29, 30, 33, 34, 38, 39, 40, 41, 42, 4...","[503, 531, 555, 520, 555, 565, 555, 506, 555, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 531, 555, 520, 555, 565, 555, 506, 555, ...","[6, 2, 0, 2, 0, 1, 5, 8, 6, 8, 9, 7, 7, 5, 1, ..."
556,NC(C(=O)OCC1OC(O)C(C(C1O)O)O)Cc1c[nH]c2c1cccc2,"[14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 3...","[503, 541, 531, 506, 531, 506, 510, 542, 507, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 541, 531, 506, 531, 506, 510, 542, 507, ...","[2, 3, 0, 2, 3, 2, 3, 5, 6, 5, 5, 0, 6, 1, 3, ..."
207,O=C(c1ccco1)N1CCN(CC1)S(=O)(=O)c1c[nH]c(=O)[nH...,"[26, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 4...","[503, 542, 510, 531, 506, 555, 520, 555, 555, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 510, 531, 506, 555, 520, 555, 555, ...","[0, 4, 3, 1, 4, 4, 5, 7, 5, 6, 7, 6, 6, 4, 3, ..."
64,OC(=O)C1OC(OC2CC(N(C2=O)C)c2cccnc2)C(C(C1O)O)O,"[17, 25, 28, 30, 37, 38, 39, 40, 41, 42, 43, 4...","[503, 542, 531, 506, 510, 542, 507, 531, 520, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[503, 542, 531, 506, 510, 542, 507, 531, 520, ...","[3, 2, 3, 4, 3, 4, 8, 6, 8, 9, 9, 9, 7, 6, 5, ..."


### Create tokenizer

In [12]:
import re
from collections import Counter

class Custom_Bart_Tokenizer():
    def __init__(self, vocab_tok_to_id: Dict[int, str] = None, unk_tok=None, pad_tok=None, eos_tok=None, bos_tok=None):
        # vocabulary for SMILES tokenization
        self.max_mz = 500 # hard coded - hyperparameter to be tested (space left for mz values, !!!!MUST corespond with the data filtering!!!!)
        
        self.tok_to_id = {}
        self.id_to_tok = {}
        self.tl_to_tok = {} # two-letter
        self.tok_to_tl = {} # two-letter
        
        self.unk_tok = unk_tok
        self.pad_tok = pad_tok
        self.eos_tok = eos_tok
        self.bos_tok = bos_tok
        self.add_special_tokens(unk_tok=unk_tok, pad_tok=pad_tok, eos_tok=eos_tok, bos_tok=bos_tok)
        
        self.first_free = 1 # the first free index (from the first 32 ASCII tokens) used to store two-letter atom names
        
    def add_token(self, token):
        if token in self.tok_to_id.keys():
            print(f"### Token {token} already present (id = {self.tok_to_id[token]})")
            return 1
        if not self.tok_to_id.values():
            new_id = self.max_mz
        else:
            new_id = sorted(self.tok_to_id.values())[-1] + 1
            print(f"setting {token} as {new_id}")
        self.tok_to_id[token] = new_id
        self.id_to_tok[new_id] = token
        
    def add_tl_token(self, token):
        if token in self.tl_to_tok.keys():
            print(f"### Two-letter token {token} already present (token = {self.tl_to_tok[token]})")
            return 1
        new_tok = chr(self.first_free) # new one letter erpresentation assigned to two-letter token
        print(f"adding {token} represented as {new_tok}")
        self.tl_to_tok[token] = new_tok
        self.tok_to_tl[new_tok] = token
        self.add_token(new_tok)
        self.first_free += 1
        
    def add_tokens(self, tok_arr):
        for tok in tok_arr:
            self.add_token(tok)
            
    def add_tl_tokens(self, tok_arr):
        for tok in tok_arr:
            self.add_tl_token(tok)
    
    def add_special_tokens(self, unk_tok=None, pad_tok=None, eos_tok=None, bos_tok=None):
        for name, var in [("unk_tok", unk_tok), ("pad_tok", pad_tok), ("eos_tok", eos_tok), ("bos_tok", bos_tok)]:
            if getattr(self, name) is not None:
                print(f"{name} already set, you can change it manually")
                continue
            if var:
                setattr(self, name, var)
                self.add_tl_token(var)
                
    
    def add_unk_token(self, name="<|ukn|>"):
        if not self.tok_to_id.values():
            new_id = self.max_mz
        else:
            new_id = sorted(self.tok_to_id.values())[-1] + 1
        # do not change defult, if it is set
        if self.add_special_tokens(unk_tok = name):
            return
        self.tok_to_id = defaultdict(lambda: new_id, self.tok_to_id)
        
    def tokenize_smiles(self, smiles):
        # SMILES -> tokens
        
        # find all known two-letter atoms and replace them with their oneletter ASCII representation
        # mrkni se, jestli tam jsou neznamy two-letter v [], prirad jim ASCII reprezentaci a nahrad je ji
        
        known_pattern = re.compile("|".join(self.tl_to_tok.keys())) # known two-letter keys in pattern
        oneletter = re.sub(known_pattern, self._assign_known, smiles)
        
        unknown_pattern = re.compile("\[[A-Z][a-z]\]") # general pattern for two-letter atoms in []
        oneletter = re.sub(unknown_pattern, self._assign_unknown, oneletter)
        
        return oneletter
    
    def detokenize(self, oneletter):
        pattern = re.compile("|".join(self.tok_to_tl.keys()))
        return re.sub(pattern, lambda x : self.tok_to_tl[x.group(0)], oneletter)
        
    
    def smiles_to_ids(self, smiles: Union[List[int], str]):
        return [self.tok_to_id[tok] for tok in self.tokenize_smiles(smiles)]
        
    def ids_to_smiles(self, ids):
        oneletter = "".join([self.id_to_tok[i] for i in ids])
        return self.detokenize(oneletter)

    def get_tokens(self):
        return self.tok_to_id.keys()
    
    def get_ids(self):
        return self.id_to_tok.keys()
    
    def get_vocab(self):
        return self.tok_to_id
    
    # fctions used in tokenize_smiles, don't use othetwise!!!
    def _assign_known(self, match):
        return self.tl_to_tok[match.group(0)]
        
    def _assign_unknown(self, match):
        tok = match.group(0)[1:3]
        self.add_tl_token(tok)
        return "[" + self.tl_to_tok[tok] +"]"

In [13]:
# otherwise all possible chars are
brackets = ["[", "]", "(", ")"]
bond_syms = [".", "-", "=", "#", "$", ":", "/", "\\"]
other_syms = ["%", "@", "+"]
nums = ["0","1","2","3","4","5","6","7","8","9"]
upper_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'] # without JQ
lower_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] # without jq
smiles_chars = brackets + bond_syms + other_syms + nums + upper_letters + lower_letters
len(smiles_chars)

73

In [14]:
tokenizer = Custom_Bart_Tokenizer()
tokenizer.add_unk_token()
tokenizer.add_special_tokens(unk_tok=None, pad_tok="<pad_tok>", eos_tok="<eos_tok>", bos_tok="<bos_tok>")
tokenizer.add_tokens(smiles_chars)
tokenizer.add_tl_tokens(["Cl", "Br"])
# tokenizer.add_tokens("Cc1nn(C)c(C)c1S(=O)(=O)N1CCN2C(=O)CN(C)C")

adding <|ukn|> represented as 
unk_tok already set, you can change it manually
adding <pad_tok> represented as 
setting  as 501
adding <eos_tok> represented as 
setting  as 502
adding <bos_tok> represented as 
setting  as 503
setting [ as 504
setting ] as 505
setting ( as 506
setting ) as 507
setting . as 508
setting - as 509
setting = as 510
setting # as 511
setting $ as 512
setting : as 513
setting / as 514
setting \ as 515
setting % as 516
setting @ as 517
setting + as 518
setting 0 as 519
setting 1 as 520
setting 2 as 521
setting 3 as 522
setting 4 as 523
setting 5 as 524
setting 6 as 525
setting 7 as 526
setting 8 as 527
setting 9 as 528
setting A as 529
setting B as 530
setting C as 531
setting D as 532
setting E as 533
setting F as 534
setting G as 535
setting H as 536
setting I as 537
setting K as 538
setting L as 539
setting M as 540
setting N as 541
setting O as 542
setting P as 543
setting R as 544
setting S as 545
setting T as 546
setting U as 547
setting V as 548
se

In [15]:
%%time
# check wether the tokenizer works
# sum([tokenizer.ids_to_smiles(tokenizer.smiles_to_ids(smiles)) == smiles for smiles in df.canon_smiles.tolist()])


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.81 µs


## Explore SMILES chars

In [16]:
from collections import Counter

In [17]:
huge_string = "".join(df["destereo_smiles"])
c = Counter(huge_string)

In [18]:
sorted(c.items(), key=lambda x: x[1], reverse=True)

[('C', 615),
 ('(', 319),
 (')', 319),
 ('O', 296),
 ('1', 240),
 ('c', 225),
 ('=', 200),
 ('N', 149),
 ('n', 111),
 ('2', 66),
 ('S', 29),
 ('[', 17),
 ('H', 17),
 (']', 17),
 ('I', 2),
 ('o', 1),
 ('s', 1),
 ('P', 1)]

In [19]:
len(c)

18

In [20]:
# num of char types found in SMILES that are not in predefined smiles_chars list
sum([key == smiles_chars for key in c.keys()])

0

In [21]:
# tokenizer.add_tokens(c.keys())

## Destereochemicalization or sth

In [22]:
#import babel

In [23]:
!ls

NIST					    dataset_old_i_guess.py
ZINC-2D-clean-instock-8.1M		    dataset_original.py
ZINC-downloader-2D-clean-instock-8.1M.curl  nist_test_smiles.sdf
ZINC-downloader-2D-smi.uri		    nist_test_smiles.txt
__pycache__				    pre-preprocess_bash_commands.txt
ales_derivatization_pipeline.ipynb	    rcx_spectra
categorized_michal.csv			    run_prepare_data.sh
concat_pickles_df.ipynb			    silicium_deriv_data_explo.ipynb
create_dataset.ipynb			    split_sdf_script.py
data_preprocess1.py			    split_sdf_script_count_mols.py
data_preprocess2.py			    trial_set
data_preprocess3.py			    visualization_preparation.ipynb
data_preprocess4.py			    visualization_preparation_df.csv
data_preprocessing.ipynb		    vizu_data.csv
data_process_stats.txt


In [24]:
# !babel -i smi -o can -xi
# remove ?stereochemics? in SMILES 
tmp_folder = "/storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/tmp" 
tmp_file = f"canon_smiles_{data_size}"
tmp_file_out = f"destereo_smiles_{data_size}"
sms = df["canon_smiles"].tolist()
with open(f"{tmp_folder}/{tmp_file}", "w+") as out:
    for s in sms:
        out.write(s + "\n")
print(f"run command: source /storage/projects/msml/mg_neims_branch/add_conda_NEIMSpy2.sh && obabel -i smi {tmp_folder}/{tmp_file} -o can -xi > {tmp_folder}/{tmp_file_out}")



KeyError: 'canon_smiles'

In [ ]:
%%time
# load the destereo smiles to the df
with open(f"{tmp_folder}/{tmp_file_out}", "r") as destereo_f:
    sms = destereo_f.readlines()
    sms = ["".join(s.split()) for s in sms] # strip whitespaces
df["destereo_smiles"] = sms

In [25]:
%%time
# tokenize the smiles
tokenized_sms = [tokenizer.smiles_to_ids(s) for s in df["destereo_smiles"]]
df["tok_smiles"] = tokenized_sms

CPU times: user 739 µs, sys: 629 µs, total: 1.37 ms
Wall time: 1.39 ms


In [26]:
# count chars in SMILES
huge_string = "".join(df["destereo_smiles"])
c = Counter(huge_string)
#sorted(c.items(), key=lambda x: x[1], reverse=True)

In [27]:
huge_int = [x for arr in df["tok_smiles"].tolist() for x in arr]
c = Counter(huge_int)
#sorted(c.items(), key=lambda x: x[0], reverse=True)

In [ ]:
# for x in df.destereo_smiles:
#     if "Cl" in x:
#         print(x)

# O=C(NC1CN(CC1O)C(=O)c1nn(c(c1Cl)C)C)Cn1ncnc1
# O=C(NC1CN(CC1O)C(=O)C1CN(C1)C(=O)C)Cn1ncc(c1)Cl  
# ClC=CCNC(=O)C(=O)NCCN1CCN(CC1)S(=O)(=O)C

In [ ]:
# SMILES length visu

import matplotlib
from matplotlib import pyplot as plt

count = Counter([len(x) for x in df.destereo_smiles])
s_count = sorted(count.items(), key=lambda x: x[0])
plt.bar([x for x,y in s_count], [y for x,y in s_count])


In [ ]:
max([len(x) for x in df.destereo_smiles])

In [ ]:
df

## Dataset creating

In [ ]:
from transformers import BartForConditionalGeneration, BartConfig
import torch

In [ ]:
SEQ_LEN = 200 # !!! MUST correspond with the data filtering hyperprams!!!

In [ ]:
%%time
# pad the tokenized smiles and mzs
df_bart = df.copy()
df_bart = df_bart[["mz", "intensity", "tok_smiles"]]
pt = tokenizer.tok_to_id[tokenizer.pad_tok]
bt = tokenizer.tok_to_id[tokenizer.bos_tok]
et = tokenizer.tok_to_id[tokenizer.eos_tok]
# for i in range(len(df)):
df_bart["tok_smiles"] = [[bt] + ts + [et] + (SEQ_LEN-2-len(ts)) * [pt] for ts in df_bart.tok_smiles]
df_bart["mz"] = [ts + (SEQ_LEN-len(ts)) * [pt] for ts in df_bart.mz]

In [ ]:
%%time
# DODELAT attention_mask
# i ostatni 
df_bart["encoder_attention_mask"] = [[int(id_ != pt) for id_ in arr] for arr in df_bart.mz]
print("DONE eam")
df_bart["decoder_attention_mask"] = [[int(id_ != pt) for id_ in arr] for arr in df_bart.tok_smiles]
print("DONE dam")
df_bart["lm_labels"] = [[id_ if int(id_ != pt) else -100 for id_ in arr] for arr in df_bart.tok_smiles]
print("DONE ll")

### Log binning

In [ ]:
%%time
base = 1.7
log_base = np.log(base)
df_bart["log_intensity"] = [np.log(arr)/log_base for arr in df_bart.intensity]

In [ ]:
%%time
# vyplotit jakou delaji standardni intensity krivku
# vyplotit jakou delaji log intensity krivku


# normal intensity
count = Counter([round(x*10) for arr in df_bart.intensity for x in arr])
s_count = sorted(count.items(), key=lambda x: x[0])
s_count_x = [x for x,y in s_count]
s_count_y = [y for x,y in s_count]
plt.bar(s_count_x, s_count_y, width=0.1)
plt.show()

In [ ]:
%%time
# log intensity
count_log = Counter([int(x) if x > -1000 else -math.inf for arr in df_bart.log_intensity for x in arr])
s_count_log = sorted(count_log.items(), key=lambda x: x[0])
s_count_log_x = [x for x,y in s_count_log]
s_count_log_y = [y for x,y in s_count_log]
s_count_log_x[0] = s_count_log_x[1] - 1
plt.bar(s_count_log_x, s_count_log_y, width=0.1)
plt.show()

In [ ]:
# plt.bar(s_count_log_x, s_count_log_y, width=0.3, log=False)
# plt.bar(s_count_x, s_count_y, width=0.01)

In [ ]:
s_count_log
s_count_log_x[1]

In [ ]:
bounds = [math.pow(1.7,(x/10)) for x in range(-0, -80, -10)]
bounds

In [ ]:
bounds.append(0.0)

In [ ]:
bounds

In [ ]:
a = np.array([1.22,3.44,6.33])#.astype(int)
(np.log(a)/log_base).astype(int) + 10

In [ ]:
%%time
# add integral bins acording to bounds (position_ids parameterzz)
log_shift = len(s_count_log)-1 # =9 for 1.7

# bacha na predelaani, pokud nebudes chtit shiftovat - je tam i ta 0!
df_bart["position_ids"] = df_bart.log_intensity.apply(lambda x: [int(n)+log_shift if n!=-math.inf else 0 for n in x])

In [ ]:
# %%time
# ASI NENI POTREBA SNAD
# add decoder_positional_ids (i think position_ids might apply on the decoder too)
# dpi = list(range(200))
# df_bart["decoder_position_ids"] = [dpi for _ in range(len(df_bart))]

In [ ]:
# len(df_bart.loc[0].intensity)
# df_bart.loc[0].mz
df_bart.head()

In [ ]:
# rename cols to match the input names
# mz -> input_ids
# tok_smiles -> decoder_input_ids
#  drop: intensities, log_intensitites

df_bart.rename(columns={"mz": "input_ids", "tok_smiles": "decoder_input_ids"}, inplace=True)
# df_bart.drop(columns=["intensity", "log_intensity"], inplace=True)
df_bart

In [ ]:
%%time
assert 0 == sum([len(x) != SEQ_LEN for x in df_bart.mz])
assert 0 == sum([len(x) != SEQ_LEN for x in df_bart.tok_smiles])
assert 0 == sum([len(x) != SEQ_LEN for x in df_bart.encoder_attention_mask])
assert 0 == sum([len(x) != SEQ_LEN for x in df_bart.decoder_attention_mask])
assert 0 == sum([len(x) != SEQ_LEN for x in df_bart.lm_labels])

In [ ]:
%%time
save_path = f"trial_set/{data_size}_bart_prepared.jsonl"
df_bart.to_json(save_path, orient="records", lines=True)

# save_path = f"trial_set/{data_size}_bart_prepared.pkl"
# df_bart.to_pickle(save_path)

## Try to build a BART and test the data

In [ ]:
config = BartConfig(vocab_size = 500+len(tokenizer.get_ids()),
                             max_position_embeddings = SEQ_LEN,
                             encoder_layers = 12,
                             encoder_ffn_dim = 4096,
                             encoder_attention_heads = 16,
                             decoder_layers = 12,
                             decoder_ffn_dim = 4096,
                             decoder_attention_heads = 16,
                             encoder_layerdrop = 0.0,
                             decoder_layerdrop = 0.0,
                             activation_function = 'gelu',
                             d_model = 1024,
                             dropout = 0.2,
                             attention_dropout = 0.0,
                             activation_dropout = 0.0,
                             init_std = 0.02,
                             classifier_dropout = 0.0,
                             scale_embedding = False,
                             use_cache = True,
                             num_labels = 3,
                             pad_token_id = 501,
                             bos_token_id = 503,
                             eos_token_id = 502,
                             is_encoder_decoder = True,
                             decoder_start_token_id = 502,
                             forced_eos_token_id = 2)
model = BartForConditionalGeneration(config)

In [ ]:
def init_trainer(args, model, train_data, valid_data, tokenizer):

#     now = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())

    training_args = TrainingArguments(
        output_dir=args.save_dir+"/"+args.save_name,         # output directory
        num_train_epochs=args.num_train_epochs,              # total # of training epochs
        per_device_train_batch_size=args.batch_size,         # batch size per device during training
        per_device_eval_batch_size=args.batch_size,          # batch size for evaluation
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        warmup_steps=args.warmup,                                    # number of warmup steps for learning rate scheduler
        weight_decay=args.weight_decay,                                   # strength of weight decay
        logging_dir=args.save_dir + './logs',                # directory for storing logs
        report_to=args.report_to,
        run_name=f"run_{now}",
        do_train=True,
        do_eval=True,
        evaluation_strategy="steps",
        eval_steps=args.eval_steps,
        logging_steps=args.log_steps,
        save_strategy="epoch",
        fp16=args.fp16,
        dataloader_drop_last=True,
        save_total_limit=2,
        dataloader_num_workers=args.num_workers,
        sharded_ddp=args.fairscale,
        deepspeed=args.deepspeed
    )

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_data,            # training dataset
        eval_dataset=valid_data,             # evaluation dataset
        data_collator = SummarizationDataCollator(tokenizer=tokenizer)
    )

    return trainer

In [ ]:
!pwd

In [ ]:
# loading the prepared data
df_path = "trial_set/1M_bart_prepared_data.pkl" 
with open(df_path, "rb") as dff:
    df = pd.read_pickle(dff)

In [15]:
df

,destereo_smiles,input_ids,decoder_input_ids,encoder_attention_mask,decoder_attention_mask,lm_labels,position_ids
0,CN1CC(=O)N2C(C1=O)CN(CC2)S(=O)(=O)c1c(C)nn(c1C)C,"[33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 48, 5...","[500, 531, 541, 520, 531, 531, 506, 510, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 531, 541, 520, 531, 531, 506, 510, 542,...","[3, 3, 1, 6, 7, 7, 9, 6, 8, 3, 2, 4, 6, 6, 6, ..."
1,OCC1CC(CN2CCOCC2)CN(C1)C(=O)CN1C(=O)CNC1=O,"[2, 30, 33, 34, 36, 38, 39, 40, 41, 42, 43, 44...","[500, 542, 531, 531, 520, 531, 531, 506, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 531, 520, 531, 531, 506, 531,...","[-9223372036854775799, 2, 4, 3, 3, 3, 6, 6, 8,..."
2,OCC1OC(Oc2ccc(cc2)NC(=O)C)C(C(C1O)O)NC(=O)C,"[14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 3...","[500, 542, 531, 531, 520, 542, 531, 506, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 531, 520, 542, 531, 506, 542,...","[3, 4, 3, 4, 4, -9223372036854775799, 3, 5, 5,..."
3,OCC1OC(Oc2ccccc2NC(=O)C)C(C(C1O)O)NC(=O)C,"[14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 3...","[500, 542, 531, 531, 520, 542, 531, 506, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 531, 520, 542, 531, 506, 542,...","[3, 5, 1, 4, 4, 3, 3, 5, 5, 6, 4, 4, 6, 2, 4, ..."
4,Fc1cccc(c1)S(=O)(=O)NCC(=O)N1CC(C1)N1C(=O)CNC1=O,"[33, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 5...","[500, 534, 555, 520, 555, 555, 555, 555, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 534, 555, 520, 555, 555, 555, 555, 506,...","[4, -9223372036854775799, 6, 6, 7, 8, 5, 7, 5,..."
...,...,...,...,...,...,...,...
509171,OCC1OC(Oc2cc(O)c(c3c2ccc(=O)o3)O)C(C(C1O)O)O,"[17, 18, 25, 26, 27, 28, 29, 30, 31, 33, 37, 3...","[500, 542, 531, 531, 520, 542, 531, 506, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 531, 520, 542, 531, 506, 542,...","[2, 3, -9223372036854775799, 2, 4, 2, 4, 3, 6,..."
509173,O=C(OC1CC(O)(C(=O)O)C(C(C1O)O)O)C=Cc1ccc(c(c1)O)O,"[16, 17, 18, 19, 25, 26, 28, 31, 32, 33, 34, 3...","[500, 542, 510, 531, 506, 542, 531, 520, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 510, 531, 506, 542, 531, 520, 531,...","[2, 5, 5, 1, 3, 3, 3, 6, 1, 2, 1, 6, 6, 7, 9, ..."
509174,OCC1OC(Oc2c(O)c(O)c(c3c2ccc(=O)o3)O)C(C(C1O)O)O,"[17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 3...","[500, 542, 531, 531, 520, 542, 531, 506, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 531, 520, 542, 531, 506, 542,...","[4, 4, 1, 3, 4, 6, 5, 6, 2, 7, 3, 1, 1, 2, 5, ..."
509175,OCC1OC(OCc2nnn(c2)c2ccc(cc2)N)C(C(C1O)O)O,"[17, 18, 26, 27, 28, 29, 30, 31, 33, 34, 36, 3...","[500, 542, 531, 531, 520, 542, 531, 506, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 531, 520, 542, 531, 506, 542,...","[4, 3, 2, 4, 6, 6, 6, 7, 5, 1, 2, 5, 6, 8, 7, ..."


### Splitting dataset

In [37]:
# splitting dataset
np.random.seed(42)
idxs = np.arange(len(df))
np.random.shuffle(idxs)
idxs

train_set_rate = 0.8
test_set_rate = 0.1
valid_set_rate = 0.1

train_len = int(len(idxs)*train_set_rate)
test_len = int((len(idxs)-train_len)*test_set_rate/(test_set_rate+valid_set_rate)) # the rest times normalized portion of test rate 
valid_len = len(idxs)-train_len-test_len # the rest

print(f"train len: {train_len}\ntest len: {test_len}\nvalid len: {valid_len}\n")
print(train_len + test_len + valid_len == len(df), ": the sum matches len of the df")

train_set = df.iloc[idxs[:train_len]]
test_set = df.iloc[idxs[train_len:train_len+test_len]]
valid_set = df.iloc[idxs[test_len:]]

# df.iloc[[1,4,5,63,2]]

train len: 354766
test len: 44346
valid len: 44346

True : the sum matches len of the df


In [42]:
# another way:
train_set_rate = 0.8
test_set_rate = 0.1
valid_set_rate = 0.1

train_set2 = df.sample(frac=train_set_rate, random_state=42)
rest = df.drop(train_set2.index)

test_set2 = rest.sample(frac=test_set_rate/(test_set_rate+valid_set_rate), random_state=42)
valid_set2 = rest.drop(test_set2.index)

print("##### SPLITTING STATS #####")
print("###########################")
print(f"train len: {len(train_set2)}\ntest len: {len(test_set2)}\nvalid len: {len(valid_set2)}\n")
print(len(train_set2) + len(test_set2) + len(valid_set2) == len(df), ": the sum matches len of the df")
print("###########################")

ValueError: Replace has to be set to `True` when upsampling the population `frac` > 1.